Dashboard Intéractif

In [ ]:
# %% [markdown]
"""
# Analyse Exploration des Données - Campagnes Commerciales
**Auteur**: Votre nom  
**Date**: DD/MM/YYYY  
**Objectif** : Identifier les leviers d'optimisation des campagnes d'appels
"""
# %%
# %% [markdown]
## 1. Chargement et Inspection des Données
# %%
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats

df = pd.read_csv('../data/generated/campaign_data.csv', parse_dates=['call_date'])
df['converted'] = df['converted'].astype(bool)

print(f"Dimensions des données : {df.shape}")
display(df.head(3))


In [ ]:

# %% [markdown]
## 2. Data Quality Check (Automatisé)
# %%
def data_quality_report(df):
    report = pd.DataFrame({
        'Type': df.dtypes,
        'Missing (%)': (df.isnull().mean() * 100).round(2),
        'Unique Values': df.nunique(),
        'Sample Values': [df[col].unique()[:3] for col in df.columns]
    })
    
    # Visualisation des valeurs manquantes
    plt.figure(figsize=(10, 4))
    sns.heatmap(df.isnull(), cbar=False)
    plt.title('Carte des Données Manquantes')
    plt.show()
    
    return report

dq_report = data_quality_report(df)
display(dq_report)


In [ ]:

# %% [markdown]
## 3. Analyse des Taux de Conversion
# %%
# Configuration globale
plt.style.use('ggplot')
conv_palette = ['#FF6B6B', '#4ECDC4']

# %% [markdown]
### 3.1 Répartition Globale
# %%
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart
df['converted'].value_counts().plot.pie(
    autopct='%1.1f%%', colors=conv_palette, ax=ax[0],
    title='Répartition Conversions', labels=['Non Converti', 'Converti']
)

# Distribution temporelle
sns.histplot(data=df, x='call_time', hue='converted', 
             multiple='stack', palette=conv_palette, ax=ax[1])
ax[1].set_title('Volume d\'Appels par Heure')
plt.tight_layout()

# %% [markdown]
### 3.2 Analyse Multidimensionnelle
# %%
def plot_interactive_sunburst():
    fig = px.sunburst(
        df,
        path=['region', 'product', 'script_version'],
        values='duration',
        color='converted',
        color_continuous_scale=conv_palette,
        title='Répartition Hiérarchique des Conversions'
    )
    fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
    fig.show()

plot_interactive_sunburst()



In [ ]:
# %% [markdown]
## 4. Analyse Temporelle Avancée
# %%
# Agrégation quotidienne
daily_data = df.groupby('call_date').agg(
    total_calls=('call_id', 'count'),
    conversion_rate=('converted', 'mean')
).reset_index()

# Découpage des tendances
result = seasonal_decompose(daily_data['conversion_rate'], model='additive', period=7)
fig = result.plot()
fig.set_size_inches(12, 8)
plt.suptitle('Décomposition des Taux de Conversion (Tendances Saisonnières)')
plt.tight_layout()



In [ ]:
# %% [markdown]
## 5. Tests Statistiques
# %%
# Test ANOVA entre scripts
script_groups = [group['converted'].values for name, group in df.groupby('script_version')]
f_val, p_val = stats.f_oneway(*script_groups)

print(f"""
🔍 Résultats des Tests Statistiques :
- ANOVA entre scripts : F-value = {f_val:.2f}, p-value = {p_val:.3f}
- Test t heures de pic vs non-pic : {stats.ttest_ind(
    df[df['call_time'].between(14,16)]['converted'],
    df[~df['call_time'].between(14,16)]['converted']
)}
""")



In [ ]:
# %% [markdown]
## 6. Analyse des Performances par Script
# %%
# Heatmap interactive
script_perf = df.pivot_table(
    index='script_version',
    columns='call_time',
    values='converted', 
    aggfunc='mean'
)

plt.figure(figsize=(10, 6))
sns.heatmap(script_perf, annot=True, fmt='.0%', cmap='YlGnBu')
plt.title('Taux de Conversion par Script et Heure')
plt.xlabel('Heure de l\'Appel')
plt.ylabel('Version du Script')



In [ ]:
# %% [markdown]
## 7. Recommandations Actionnables
# %%
# Calcul des meilleures combinaisons
best_combinations = df.groupby(['script_version', 'call_time', 'product']).agg(
    conversion_rate=('converted', 'mean'),
    volume=('call_id', 'count')
).sort_values('conversion_rate', ascending=False).head(5)

print("Top 5 des Combinaisons Performantes :")
display(best_combinations.style.format({
    'conversion_rate': '{:.1%}',
    'volume': '{:,}'
}).background_gradient(cmap='Blues'))


In [ ]:

# %% [markdown]
## 8. Export des Insights
# %%
# Génération automatique de slides
from pptx import Presentation
from pptx.util import Inches

prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[5])

# Ajout du titre
title = slide.shapes.title
title.text = "Principaux Insights - Campagnes Commerciales"

# Ajout d'un graphique
img_path = 'conversion_heatmap.png'
plt.savefig(img_path, bbox_inches='tight')
left = top = Inches(1)
slide.shapes.add_picture(img_path, left, top, height=Inches(4.5))

prs.save('campaign_insights.pptx')